In [1]:
import altair as alt
from altair import Chart, X, Y, Color, Scale, Axis
import pandas as pd

import gif

In [2]:
df = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/full_data.csv')
df['date'] = pd.to_datetime(df['date'])

In [3]:
select = df[df['location'].isin([
    'United States',
    'China',
    'Canada',
    'Sweden']
)].reset_index(drop=True)

In [4]:
population = {
    'United States': 330_000_000,
    'China': 1_440_000_000,
    'Canada': 37_000_000,
    'Sweden': 10_000_100
}

In [5]:
select['population'] = select['location'].map(population)
select['cases_per_million'] = select.apply(lambda row: row['new_cases'] / row['population'] * 1_000_000, axis=1)
select['7d'] = select.groupby('location').rolling(7)['cases_per_million'].mean().reset_index(drop=True)

In [6]:
@gif.frame
def plot(date):
    
    d = select[select['date'] <= date]
    xmin = d['date'].min()
    delta = (d['date'].max() - xmin).days    
    tmax = d['date'].max() + pd.Timedelta(days=max(30, delta))
    xmax = min(tmax, d['date'].max()) + pd.Timedelta(days=5)
    ymax = max(5, d['7d'].max()) + 5
    date_ticks = pd.date_range(xmin, pd.Timestamp('now'), freq='MS').tolist()
    
    chart = Chart(d).encode(
        x=X("date", 
            title=None, 
            scale=Scale(domain=(xmin, xmax)), 
            axis=Axis(format="%b", values=date_ticks)
        ),
        y=Y('7d', title=None, scale=Scale(domain=(0, ymax))),
        color=Color('location', 
            scale=Scale(
                domain=['United States', 'China', 'Canada', 'Sweden'], 
                range=["#000055", "#550000", "#881111", "#2222CC"]
            ),
            legend=None
        )
    ).mark_line().properties(width=500, height=300)
    
    return chart

In [ ]:
dates = pd.date_range(start='2020-02-01', end=df['date'].max())
frames = []
for date in dates:
    frame = plot(date)
    frames.append(frame)

gif.save(frames, 'covid.gif', duration=75)